In [6]:
import datetime
import numpy as np
import pprint
import time

from dateutil import parser
import fiona
import ipywidgets as widgets
import pandas as pd
import pygsheets
import rtree
from tqdm._tqdm_notebook import tqdm_notebook
import shapely.geometry

creds = pygsheets.authorize(service_file='/home/thejakeschmidt/311-open-data-analysis/client_secret.json')

SUMMONS_DESCRIPTION = 'The Police Department issued a summons in response to the complaint.'
ACTION_DESCRIPTION = 'The Police Department responded to the complaint and took action to fix the condition.'

# Read data and add calculated columns

In [7]:
def read_raw_data():
    # illegal_parking.csv is the data from
    # https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/7ahn-ypff, manually filtered to the rows containing the string "Illegal Parking".
    data = pd.read_csv('data/illegal_parking.csv')
    series = data.groupby('Resolution Description').size().sort_values(ascending=False)
    series.name = 'count'
    return data

In [8]:
def add_date_and_time_columns(data):
    data['Created Date'] = pd.to_datetime(data['Created Date'])
    data['Calculated Year'] = data['Created Date'].map(lambda date: date.year)
    data['Calculated Closed Date'] = data['Closed Date'].apply(
        lambda datestr:
        np.datetime64('NaT') if (pd.isnull(datestr) or datestr == '01/01/1900 12:00:00 AM')
        else parser.parse(datestr))
    data['Calculated Resolution Time'] = data['Calculated Closed Date'] - data['Created Date']

In [9]:
def add_community_board_column(data):
    """The raw "Community Board" column is formatted like "01 MANHATTAN". This adds one formatted like "Manhattan CB1"."""
    def reformat_community_board(cb):
        parts = cb.split(' ', 1)
        return parts[1].title() + " CB" + (str(int(parts[0])) if parts[0] != 'Unspecified' else parts[0])
    data['Calculated Community Board'] = data['Community Board'].map(reformat_community_board)

In [10]:
def add_borough_column(data):
    # The Borough field is mostly empty, so we have to get that from the zip codes                                                                                                                                 
    bronx_zips = {
        10451, 10452, 10453, 10454, 10455, 10456, 10457, 10458, 10459, 10460, 10461, 10462, 10463, 10464, 10465, 10466,
        10467, 10468, 10469, 10470, 10471, 10472, 10473, 10474, 10475}
    brooklyn_zips = {
        11201, 11203, 11204, 11205, 11206, 11207, 11208, 11209, 11210, 11211, 11212, 11213, 11214, 11215, 11216, 11217,
        11218, 11219, 11220, 11221, 11222, 11223, 11224, 11225, 11226, 11228, 11229, 11230, 11231, 11232, 11233, 11234,
        11235, 11236, 11236, 11237, 11237, 11238, 11238, 11239, 11239, 11241, 11241, 11242, 11242, 11243, 11243, 11249,
        11249, 11252, 11252, 11256}
    manhattan_zips = {
        10001, 10002, 10003, 10004, 10005, 10006, 10007, 10009, 10010, 10011, 10012, 10013, 10014, 10015, 10016, 10017,
        10018, 10019, 10020, 10021, 10022, 10023, 10024, 10025, 10026, 10027, 10028, 10029, 10030, 10031, 10032, 10033,
        10034, 10035, 10036, 10037, 10038, 10039, 10040, 10041, 10044, 10045, 10048, 10055, 10060, 10069, 10090, 10095,
        10098, 10099, 10103, 10104, 10105, 10106, 10107, 10110, 10111, 10112, 10115, 10118, 10119, 10120, 10121, 10122,
        10123, 10128, 10151, 10152, 10153, 10154, 10155, 10158, 10161, 10162, 10165, 10166, 10167, 10168, 10169, 10170,
        10171, 10172, 10173, 10174, 10175, 10176, 10177, 10178, 10199, 10270, 10271, 10278, 10279, 10280, 10281, 10282}
    queens_zips = {
        11004, 11101, 11102, 11103, 11104, 11105, 11106, 11109, 11351, 11354, 11355, 11356, 11357, 11358, 11359, 11360,
        11361, 11362, 11363, 11364, 11365, 11366, 11367, 11368, 11369, 11370, 11371, 11372, 11373, 11374, 11375, 11377,
        11378, 11379, 11385, 11411, 11412, 11413, 11414, 11415, 11416, 11417, 11418, 11419, 11420, 11421, 11422, 11423,
        11426, 11427, 11428, 11429, 11430, 11432, 11433, 11434, 11435, 11436, 11691, 11692, 11693, 11694, 11697}
    staten_island_zips = {
        10301, 10302, 10303, 10304, 10305, 10306, 10307, 10308, 10309, 10310, 10311, 10312, 10314}
    borough_mapping = {}
    borough_mapping.update({zip: 'Bronx' for zip in bronx_zips})
    borough_mapping.update({zip: 'Brooklyn' for zip in brooklyn_zips})
    borough_mapping.update({zip: 'Manhattan' for zip in manhattan_zips})
    borough_mapping.update({zip: 'Queens' for zip in queens_zips})
    borough_mapping.update({zip: 'Staten Island' for zip in staten_island_zips})
    data['Calculated Borough'] = data['Incident Zip'].map(lambda zip: borough_mapping[zip] if zip in borough_mapping else 'Unknown')

In [12]:
# From https://data.cityofnewyork.us/Public-Safety/Police-Precincts/78dh-3ptz
precincts_shapefile = '/home/thejakeschmidt/311-open-data-analysis/illegal_parking/data/precinct_shapes/geo_export_1d0f5023-ecf5-4d19-833e-83428f65f9ff.shp'

# Copied from https://stackoverflow.com/a/20007730
def ordinal(n):
    return '%d%s' % (n, 'tsnrhtdd'[(n/10%10!=1)*(n%10<4)*n%10::4])

def get_precinct_name(raw_precinct):
    if raw_precinct == 14:
        return 'Midtown South'
    if raw_precinct == 18:
        return 'Midtown North'
    if raw_precinct == 22:
        return 'Central Park'
    return ordinal(raw_precinct)

def get_precinct_for_point(precincts_rtree, precinct_shapes, precinct_names, lat, lng):
    if pd.isnull(lat) or pd.isnull(lng):
        return 'unknown'
    point = shapely.geometry.Point(lng, lat)
    containing_precincts = [p for p in precincts_rtree.intersection((lng, lat))
                            if precinct_shapes[p].contains(point)]
    if len(containing_precincts) > 1:
        raise Exception('Found multiple precincts for point (' + str(lat) + ', ' + str(lng) + '): ' + str(containing_precincts))
    if len(containing_precincts) == 0:
        return 'unknown'
    return precinct_names[containing_precincts[0]]

def get_precinct_names_and_numbers():
    precinct_names_and_numbers = []
    for i, precinct_shape in enumerate(fiona.open(precincts_shapefile)):
        precinct_number = int(precinct_shape['properties']['precinct'])
        precinct_names_and_numbers.append((get_precinct_name(precinct_number), precinct_number))
    return precinct_names_and_numbers

precincts_sorting_order = {name: i for name, i in get_precinct_names_and_numbers()}
unknown_precinct_sorting_order = max([i for _, i in precincts_sorting_order.iteritems()]) + 1
def get_precinct_sorting_order(precinct_name):
    if precinct_name in precincts_sorting_order:
        return precincts_sorting_order[precinct_name]
    return unknown_precinct_sorting_order

def add_precinct_column(data):
    precinct_names = []
    precinct_shapes = []
    precincts_rtree = rtree.index.Index()
    for i, precinct_shape in enumerate(fiona.open(precincts_shapefile)):
        precinct_names.append(get_precinct_name(int(precinct_shape['properties']['precinct'])))
        precinct_shapes.append(shapely.geometry.asShape(precinct_shape['geometry']))
        precincts_rtree.insert(i, shapely.geometry.asShape(precinct_shape['geometry']).bounds)    

    # Optional progress bar support; disable by commenting out this line and changing progress_apply to apply
    tqdm_notebook().pandas(desc="Adding precincts")
    data['Calculated Precinct'] = data[['Latitude', 'Longitude']].progress_apply(
        lambda row: get_precinct_for_point(
            precincts_rtree, precinct_shapes, precinct_names, row['Latitude'], row['Longitude']), axis=1)

In [13]:
# From https://data.cityofnewyork.us/City-Government/City-Council-Districts/yusd-j4xi
council_districts_shapefile = '/home/thejakeschmidt/311-open-data-analysis/illegal_parking/data/council_district_shapes/geo_export_16ddf551-ecd0-45f5-9681-5b2046184eb1.shp'

# Theoretically available at https://data.cityofnewyork.us/City-Government/Council-Members/uvw5-9znb,
# but that's out of date as of 2018-01-25. I made my own with the two columns I need.
council_members_raw = pd.read_csv('data/Council_Members.csv')
council_members = list(council_members_raw.apply(lambda row: (row['NAME'], row['DISTRICT']), axis=1))
council_members_by_district = {district: name for name, district in council_members}

def get_council_district_name(council_district):
    return '{} ({})'.format(council_district, council_members_by_district[council_district])

def get_council_district_for_point(council_districts_rtree, council_district_shapes, lat, lng):
    if pd.isnull(lat) or pd.isnull(lng):
        return 'unknown'
    point = shapely.geometry.Point(lng, lat)
    containing_council_districts = [
        d for d in council_districts_rtree.intersection((lng, lat))
        if council_district_shapes[d].contains(point)]
    if len(containing_council_districts) > 1:
        raise Exception(
            'Found multiple council districts for point (' + str(lat) + ', ' + str(lng) + '): ' +
            str(containing_council_districts))
    if len(containing_council_districts) == 0:
        return 'unknown'
    return get_council_district_name(containing_council_districts[0])

def add_council_district_column(data):
    council_district_shapes = {}
    council_districts_rtree = rtree.index.Index()
    for council_district_shape in fiona.open(council_districts_shapefile):
        council_district = int(council_district_shape['properties']['coun_dist'])
        council_district_shapes[council_district] = shapely.geometry.asShape(council_district_shape['geometry'])
        council_districts_rtree.insert(
            council_district, shapely.geometry.asShape(council_district_shape['geometry']).bounds)

    # Optional progress bar support; disable by commenting out this line and changing progress_apply to apply
    tqdm_notebook().pandas(desc="Adding council districts")
    data['Calculated Council District'] = data[['Latitude', 'Longitude']].progress_apply(
        lambda row: get_council_district_for_point(
            council_districts_rtree, council_district_shapes, row['Latitude'], row['Longitude']), axis=1)

In [494]:
# Regenerate enhanced data (only needed if definitions change)

# data = read_raw_data()
# add_community_board_column(data)
# add_date_and_time_columns(data)
# add_borough_column(data)
# add_summons_column(data)
# add_action_column(data)
# add_precinct_column(data) # Warning: takes about 2-3 hours on my computer
# add_council_district_column(data) # Warning: takes about 2-3 hours on my computer
# data = data.rename(columns={'Council District': 'Calculated Council District'})
# data.to_pickle('data/enhanced_data.pkl')

In [14]:
# Load from file
data = pd.read_pickle('data/enhanced_data.pkl')

# Calculating 2d breakdowns
##### These calculate fractions of successful outcomes (action, summons, or both) by two breakdown columns (e.g. community board and descriptor), and output CSV files. They present a lot of data densely, but aren't as easy to read.

In [194]:
def get_2d_outcome_fraction(data, breakdown_column1, breakdown_column2, outcome_column, outcome_values):
    """Groups data by two breakdown columns, with values being the fraction of specified values in an outcome column."""
    data = data.copy()
    data['synthesized_outcome_value'] = data[outcome_column].isin(outcome_values)
    outcome_counts = (data.groupby([breakdown_column1, breakdown_column2, 'synthesized_outcome_value'])
                       .size().unstack().fillna(0))
    return (outcome_counts[True] / (outcome_counts[True] + outcome_counts[False])).unstack()

In [37]:
# Generate by-borough CSVs
get_2d_outcome_fraction(
    data, 'Descriptor', 'Calculated Borough', 'Resolution Description', [SUMMONS_DESCRIPTION])\
.to_csv('out/summons_by_borough_and_descriptor.csv')
get_2d_outcome_fraction(
    data, 'Descriptor', 'Calculated Borough', 'Resolution Description', [ACTION_DESCRIPTION])\
.to_csv('out/action_by_borough_and_descriptor.csv')
get_2d_outcome_fraction(
    data, 'Descriptor', 'Calculated Borough', 'Resolution Description', [SUMMONS_DESCRIPTION, ACTION_DESCRIPTION])\
.to_csv('out/positive_outcome_by_borough_and_descriptor.csv')

In [38]:
# Generate by-CB CSVs
get_2d_outcome_fraction(
    data.loc[~data['Calculated Community Board'].str.contains('Unspecified')].copy(),
    'Descriptor', 'Calculated Community Board', 'Resolution Description', [SUMMONS_DESCRIPTION])\
.to_csv('out/summons_by_cb_and_descriptor.csv')
get_2d_outcome_fraction(
    data.loc[~data['Calculated Community Board'].str.contains('Unspecified')].copy(),
    'Descriptor', 'Calculated Community Board', 'Resolution Description', [ACTION_DESCRIPTION])\
.to_csv('out/action_by_cb_and_descriptor.csv')
get_2d_outcome_fraction(
    data.loc[~data['Calculated Community Board'].str.contains('Unspecified')].copy(),
    'Descriptor', 'Calculated Community Board', 'Resolution Description', [SUMMONS_DESCRIPTION, ACTION_DESCRIPTION])\
.to_csv('out/positive_outcome_by_cb_and_descriptor.csv')

In [85]:
# Generate by-precinct CSVs
get_2d_outcome_fraction(
    data, 'Descriptor', 'Calculated Precinct', 'Resolution Description', [SUMMONS_DESCRIPTION])\
.to_csv('out/summons_by_precinct_and_descriptor.csv')
get_2d_outcome_fraction(
    data, 'Descriptor', 'Calculated Precinct', 'Resolution Description', [ACTION_DESCRIPTION])\
.to_csv('out/action_by_precinct_and_descriptor.csv')
get_2d_outcome_fraction(
    data, 'Descriptor', 'Calculated Precinct', 'Resolution Description', [SUMMONS_DESCRIPTION, ACTION_DESCRIPTION])\
.to_csv('out/positive_outcome_by_precinct_and_descriptor.csv')

# Calculating 1d breakdowns
##### These calculate the number and fraction of outcomes (action, summons, and other) broken down by one dimension, for only a single descriptor (e.g. "Blocked Bike Lane" or "Blocked Sidewalk"), and writes them to Google Sheets. These are easier to read and more actionable.

In [15]:
def generate_1d_outcome_breakdown(data, descriptor, breakdown_column, breakdown_sorting_key=lambda v: v):
    outcome_mapping = {ACTION_DESCRIPTION: '"Took action"',
                       SUMMONS_DESCRIPTION: '"Issued summons"'}

    filtered_data = data.loc[(data['Descriptor'] == descriptor)].copy()
    filtered_data['outcome'] = filtered_data['Resolution Description'].apply(
        lambda desc: outcome_mapping[desc] if desc in outcome_mapping else 'None')

    by_outcome = filtered_data\
     .groupby([breakdown_column, 'outcome'])\
     .size()\
     .unstack()\
     .fillna(0)

    by_outcome[breakdown_column] = by_outcome.index
    by_outcome['sorting_key'] = by_outcome[breakdown_column].apply(breakdown_sorting_key)
    by_outcome.sort_values(by=['sorting_key'], inplace=True)
    by_outcome.drop('sorting_key', axis=1, inplace=True)

    by_outcome['Total reports'] = by_outcome[['"Took action"', '"Issued summons"', 'None']].sum(axis=1)
    return by_outcome[['"Took action"', '"Issued summons"', 'Total reports']]

In [16]:
def generate_cb_breakdown(data, descriptor):
    def get_sorting_key(cb):
        parts = cb.rsplit(' ', 1)
        return (parts[0], int(parts[1][2:]))
    return generate_1d_outcome_breakdown(
        data.loc[~data['Calculated Community Board'].str.contains('Unspecified')],
        descriptor, 'Calculated Community Board', get_sorting_key)

In [17]:
def generate_borough_breakdown(data, descriptor):
    return generate_1d_outcome_breakdown(data, descriptor, 'Calculated Borough')

In [18]:
def generate_precinct_breakdown(data, descriptor):
    return generate_1d_outcome_breakdown(data, descriptor, 'Calculated Precinct', get_precinct_sorting_order)

In [19]:
def generate_council_district_breakdown(data, descriptor):
    def get_sorting_key(council_district):
        if council_district == 'unknown':
            return 99999
        return int(council_district.split(' ')[0])
    return generate_1d_outcome_breakdown(data, descriptor, 'Calculated Council District', get_sorting_key)

In [20]:
# These generate CSVs with the 1d breakdown data. Don't bother with these if you can get
# write_1d_breakdown_to_sheet working for you, it generates more detailed spreadsheets.

# def generate_by_cb_and_by_precinct_csvs(data, descriptor, csv_name_fragment):
#     generate_cb_breakdown(data, descriptor).to_csv('out/' + csv_name_fragment + '_by_cb.csv')
#     generate_precinct_breakdown(data, descriptor).to_csv('out/' + csv_name_fragment + '_by_precinct.csv')

# generate_by_cb_and_by_precinct_csvs(data, 'Blocked Bike Lane', 'blocked_bike_lane')
# generate_by_cb_and_by_precinct_csvs(data, 'Blocked Crosswalk', 'blocked_crosswalk')
# generate_by_cb_and_by_precinct_csvs(data, 'Blocked Sidewalk', 'blocked_sidewalk')
# generate_by_cb_and_by_precinct_csvs(data, 'Parking Permit Improper Use', 'parking_permit_improper_use')
# generate_by_cb_and_by_precinct_csvs(data, 'Posted Parking Sign Violation', 'posted_parking_sign_violation')

In [21]:
def add_conditional_formatting_rule(spreadsheet, worksheet_index, column, midpoint_value):
    request = {
        'addConditionalFormatRule': {
            'rule': {
                'ranges': [{
                    'sheetId': spreadsheet[worksheet_index].id,
                    'startRowIndex': 1,
                    'endRowIndex': 100,
                    'startColumnIndex': column,
                    'endColumnIndex': column+1
                }],
                'gradientRule': {
                    'minpoint': {
                        # light red berry 2
                        'color': {
                            'red': 0.866,
                            'green': 0.494,
                            'blue': 0.420
                        },
                        'type': 'PERCENTILE',
                        'value': '5'
                    },
                    'midpoint': {
                        'color': {
                            'red': 1,
                            'green': 1,
                            'blue': 1
                        },
                        'type': 'NUMBER',
                        'value': str(midpoint_value)
                    },
                    'maxpoint': {
                        # light green 2
                        'color': {
                            'red': 0.714,
                            'green': 0.843,
                            'blue': 0.659
                        },
                        'type': 'PERCENTILE',
                        'value': '95'
                    },
                }
            },
            'index': 0
        }
    }
    # This is hitting an internal-ish pygsheets API, which probably isn't very stable over time.
    spreadsheet.client.sheet.batch_update(spreadsheet.id, request, fields='*')

In [22]:
def write_1d_breakdown_to_sheet(
        df, spreadsheet, worksheet_index, title, breakdown_column_display_name,
        breakdown_column_width):
    """Takes a dataframe returned from generate_1d_outcome_breakdown and writes it to a Google Sheet."""
    # 1a. Add extra columns for percentages
    df = df.copy()
    breakdown_column = df.index.name
    df = df.reset_index(drop=False).copy()
    
    total_actions = df['"Took action"'].sum()
    total_summons = df['"Issued summons"'].sum()
    total_reports = df['Total reports'].sum()
    
    # 1b. Insert a line for the totals
    total_line = pd.DataFrame(
        {breakdown_column: '',
         '"Took action"': '=SUM(B3:B{})'.format(len(df) + 2),
         '"Issued summons"': '=SUM(D3:D{})'.format(len(df) + 2),
         'Total reports': '=SUM(F3:F{})'.format(len(df) + 2)},
        index=['Total'], columns=df.columns)
    df = pd.concat([total_line, df], sort=False).reset_index(drop=True)

    # 1c. Add percentage columns and put everything in order
    df['"Took action" %'] = pd.Series(['=B{}/F{}'.format(i + 2, i + 2) for i in range(len(df))])
    df['"Issued summons" %'] = pd.Series(['=D{}/F{}'.format(i + 2, i + 2) for i in range(len(df))])
    df = df[[breakdown_column, '"Took action"', '"Took action" %', '"Issued summons"', '"Issued summons" %', 'Total reports']]
    
    # 2. Write to spreadsheet
    
    sheet = spreadsheet[worksheet_index]
    
    sheet.set_dataframe(df, (1, 1))
    sheet.cell('A1').value = breakdown_column_display_name

    bold_cell = pygsheets.Cell('A1', '')
    bold_cell.set_text_format('bold', True)
    sheet.range('A1:F2', returnas='range').apply_format(bold_cell)

    percent_cell = pygsheets.Cell('A1', '')
    percent_cell.set_number_format(pygsheets.FormatType.PERCENT, '0.0%')
    sheet.range('C2:C{}'.format(len(df)+2), returnas='range').apply_format(percent_cell)
    sheet.range('E2:E{}'.format(len(df)+2), returnas='range').apply_format(percent_cell)

    # The DataRange.apply_format overwrites the bold, so we have to redo that for the
    # cells that are bold percentages.
    sheet.cell('C2').set_text_format('bold', True)
    sheet.cell('E2').set_text_format('bold', True)
    
    sheet.frozen_rows = 2
    sheet.frozen_cols = 1
    
    sheet.adjust_column_width(0, pixel_size=breakdown_column_width)
    sheet.adjust_column_width(1, pixel_size=91)
    sheet.adjust_column_width(2, pixel_size=106)
    sheet.adjust_column_width(3, pixel_size=126)
    sheet.adjust_column_width(4, pixel_size=141)
    sheet.adjust_column_width(5, pixel_size=109)

    sheet.title = title

    add_conditional_formatting_rule(spreadsheet, worksheet_index, 2, float(total_actions) / total_reports)
    add_conditional_formatting_rule(spreadsheet, worksheet_index, 4, float(total_summons) / total_reports)

In [32]:
def write_breakdown_sheets(data, descriptor):
    # Note: you have to create a spreadsheet with the right name first.
    spreadsheet = creds.open('311 "{}" reports'.format(descriptor))

    # WARNING: This clears out all existing worksheets!
    if not any([worksheet.title == 'Loading...' for worksheet in spreadsheet.worksheets()]):
        spreadsheet.add_worksheet('Loading...')
    # pygsheets doesn't seem to like when you delete worksheets while iterating over .worksheets(),
    # so we have to do it a little more awkwardly.
    while True:
        non_loading_worksheets = [
            worksheet for worksheet in spreadsheet.worksheets() if worksheet.title != 'Loading...']
        if len(non_loading_worksheets) > 0:
            spreadsheet.del_worksheet(non_loading_worksheets[0])
        else:
            break
    spreadsheet[0].clear()

    data_2018 = data.loc[data['Calculated Year'] == 2018]
    
    about_worksheet = spreadsheet[0]
    about_worksheet.cell('A1').value = (
        'This spreadsheet breaks down the "{}" reports sent to 311 from 2010-2018. The worksheet tabs ' +
        'show the available breakdowns: by borough, by community board, by NYPD precinct, ' +
        'and by City Council district. Each breakdown is done once with all available data, and once ' +
        'with 2018 data only.').format(descriptor)
    about_worksheet.cell('A2').value = (
        'The code that generates these spreadsheets is available at ' +
        'https://github.com/TheJakeSchmidt/311-open-data-analysis, and the author is at ' +
        'https://twitter.com/TheJakeSchmidt. Feel free to tweet questions or requests at me, ' +
        'and I\'ll see what I can do!')
    about_worksheet.adjust_column_width(0, pixel_size=600)
    about_worksheet.adjust_column_width(0, pixel_size=600)
    about_worksheet.cell('A1').wrap_strategy = 'WRAP'
    about_worksheet.cell('A2').wrap_strategy = 'WRAP'
    about_worksheet.title = 'About'

    spreadsheet.add_worksheet('Loading...')
    write_1d_breakdown_to_sheet(
        generate_borough_breakdown(data_2018, descriptor),
        spreadsheet, 1, 'By borough (2018)', 'Borough', 83)

    spreadsheet.add_worksheet('Loading...')
    write_1d_breakdown_to_sheet(
        generate_borough_breakdown(data, descriptor),
        spreadsheet, 2, 'By borough (2010-2018)', 'Borough', 83)

    spreadsheet.add_worksheet('Loading...')
    write_1d_breakdown_to_sheet(
        generate_cb_breakdown(data_2018, descriptor),
        spreadsheet, 3, 'By community board (2018)', 'Community board', 118)

    spreadsheet.add_worksheet('Loading...')
    write_1d_breakdown_to_sheet(
        generate_cb_breakdown(data, descriptor),
        spreadsheet, 4, 'By community board (2010-2018)', 'Community board', 118)

    spreadsheet.add_worksheet('Loading...')
    write_1d_breakdown_to_sheet(
        generate_precinct_breakdown(data_2018, descriptor),
        spreadsheet, 5, 'By NYPD precinct (2018)', 'Precinct', 93)

    spreadsheet.add_worksheet('Loading...')
    write_1d_breakdown_to_sheet(
        generate_precinct_breakdown(data, descriptor),
        spreadsheet, 6, 'By NYPD precinct (2010-2018)', 'Precinct', 93)

    spreadsheet.add_worksheet('Loading...')
    write_1d_breakdown_to_sheet(
        generate_council_district_breakdown(data_2018, descriptor),
        spreadsheet, 7, 'By city council district (2018)', 'Council district', 159)

    spreadsheet.add_worksheet('Loading...')
    write_1d_breakdown_to_sheet(
        generate_council_district_breakdown(data, descriptor),
        spreadsheet, 8, 'By city council district (2010-2018)', 'Council district', 159)

In [33]:
write_breakdown_sheets(data, 'Blocked Bike Lane')
#write_breakdown_sheets(data, 'Blocked Sidewalk')
#write_breakdown_sheets(data, 'Parking Permit Improper Use')
#write_breakdown_sheets(data, 'Posted Parking Sign Violation')

# Calculating time-based breakdowns

In [576]:
with_minutes = data.loc[~data['Calculated Resolution Time'].isnull()].copy()
with_minutes['Calculated Resolution Minutes'] = ((
    (with_minutes['Calculated Resolution Time'] // np.timedelta64(15, 'm')).astype('int') * 15)
    .clip(upper=1440))
generate_1d_outcome_breakdown(with_minutes, 'Blocked Bike Lane', 'Calculated Resolution Minutes')\
.reindex(pd.Index(range(0, 1441, 15))).to_csv('out/by_resolution_minutes.csv')
# Interesting: no apparent correlation; summons and actions are both pretty consistent for the first 10-12 hours.
# Small downward trend in actions over the first 8 hours, but really not much. Hardly explanatory.